In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# Vespa - Application Package

> Python API to create, modify and deploy application packages

Our goal is to create, modify and deploy simple application packages using our python API. This enables us to run data analysis experiments that are fully integrated with Vespa. As an example, we want to create the application package we used in our [text search tutorial](https://docs.vespa.ai/documentation/tutorials/text-search.html). 

## Create schema

Our goal in this section is to create the following `msmarco` schema using our python API.

```
schema msmarco {
    document msmarco {
        field id type string {
            indexing: attribute | summary
        }
        field title type string {
            indexing: index | summary
            index: enable-bm25
        }
        field body type string {
            indexing: index | summary
            index: enable-bm25
        }
    }

    fieldset default {
        fields: title, body
    }

    rank-profile default {
        first-phase {
            expression: nativeRank(title, body)
        }
    }

    rank-profile bm25 inherits default {
        first-phase {
            expression: bm25(title) + bm25(body)
        }
    }

}
```

In [ ]:
document = Document()
document.add_field(
    Field(name = "id", type = "string", indexing = ["attribute", "summary"]),
    Field(name = "title", type = "string", indexing = ["index", "summary"], index = ["enable-bm25"]),
    Field(name = "body", type = "string", indexing = ["index", "summary"], index = ["enable-bm25"])
)

default_fieldset = FieldSet(name = "default", fields = ["title", "body"])

default_rank_profile = RankProfile(name = "default", first_phase = "nativeRank(title, body)")

msmarco_schema = Schema(
    name = "msmarco", 
    document = document, 
    default_fieldset = default_fieldset,
    default_rank_profile = default_rank_profile
)

msmarco_schema.add_rank_profile(
    RankProfile(name = "bm25", inherits = "default", first_phase = "bm25(title) + bm25(body)")
)

## Create application package

In [ ]:
application_package = ApplicationPackage(name = "msmarco_app")
application_package.add_schema(msmarco_schema)

## Deploy application package

In [ ]:
application_package.deploy()